In [1]:
!pip install cripser
!pip install tcripser

     ---------------------------------------- 0.0/53.4 kB ? eta -:--:--
     ---------------------------------------- 53.4/53.4 kB 1.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/179.2 kB ? eta -:--:--
   ------------- -------------------------- 61.4/179.2 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 179.2/179.2 kB 1.8 MB/s eta 0:00:00


ERROR: Could not find a version that satisfies the requirement tcripser (from versions: none)
ERROR: No matching distribution found for tcripser


In [2]:
import os
import re
import numpy as np
from PIL import Image
from scipy.ndimage import distance_transform_edt
from skimage.filters import threshold_otsu  # Add this import statement
import cripser as cr
import tcripser as tcr
import persim
from persim.persistent_entropy import *
import pandas as pd

data=[]

numbers = re.compile(r'(\d+)')
def numericalSort(value):
    parts = numbers.split(value)
    parts[1::2] = map(int, parts[1::2])
    return parts

# Function to compute distance transform
def dt(img, radius=15, signed=False):
    bw_img = (img >= threshold_otsu(img))
    dt_img = distance_transform_edt(bw_img)
    if signed:
        dt_img -= distance_transform_edt(~bw_img)
    return dt_img

# Function to compute persistence diagram and calculate persistence entropy
def compute_persistence_entropy(imgfile):
    # Load image data
    img2d = np.array(Image.open(imgfile).convert('L'))

    # Apply distance transform
    img2d_dt = dt(img2d)

    # Compute persistence diagrams
    pd = cr.computePH(img2d)
    pds = [pd[pd[:, 0] == i] for i in range(3)]

    # Compute persistence entropy
    entropies = []
    for p in pds:
        entropy = persistent_entropy(p[:, 1:3])
        entropies.append(entropy)

    return entropies

if __name__ == "__main__":
    # Provide the path to the input DICOM folder
    class_labels=["Bengin cases","Normal cases","Malignant cases"]
    for class_label in class_labels:
        input_folder = "C:\\Users\\Aneesh PB\\Downloads\\Topology Project\\Full data"
        input_folder = os.path.join(input_folder, class_label)
        #class_labels=["Benign","Normal","Malignant"]
        # Iterate through each image in the folder
        for imgfile in sorted(os.listdir(input_folder), key=numericalSort):
            imgfile = os.path.join(input_folder, imgfile)
            if os.path.isfile(imgfile):
                entropies = compute_persistence_entropy(imgfile)
                entropy_0=entropies[0][0]*10**301
                entropy_1=entropies[1][0]
                # print(f"Entropies for {imgfile}:",entropy_0,entropy_1)
                data.append({'Entropy 0': entropy_0, 'Entropy 1': entropy_1, 'Class': class_label})
df = pd.DataFrame(data)
df.to_excel("C:\\Users\\Aneesh PB\\Downloads\\Topology Project\\VS code\\entropy.xlsx", index=False)
# Append data to existing Excel file or create a new one if it doesn't exist


In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix

# Step 1: Data Preprocessing
df =pd.read_excel("C:\\Users\\Aneesh PB\\Downloads\\Topology Project\\VS code\\entropy.xlsx")
classes = ['Normal cases', 'Bengin cases', 'Malignant cases']

X=df.iloc[:,:2]
y=df.iloc[:,2]
X
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)



In [4]:
X

,Entropy 0,Entropy 1
0,3.591387,8.971071
1,3.433466,8.937837
2,3.185794,8.886192
3,2.772464,8.979550
4,2.780078,8.962683
...,...,...
1092,0.643065,7.785089
1093,0.641343,7.722035
1094,0.647753,7.671827
1095,0.628998,7.672080


In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

le=LabelEncoder()
y_train=le.fit_transform(y_train)
y_test=le.fit_transform(y_test)
# Define the range of random states to test
random_states = range(0, 50)  # Adjust this range as needed

# Define the models with initial parameters
models = {
    'Random Forest': RandomForestClassifier(n_jobs=-1),
    'XGBoost': XGBClassifier(n_jobs=-1),  # Assuming best_params are defined
    'SVM': SVC(),
    'KNN': KNeighborsClassifier(),
}

# Dictionary to store the best random state and accuracy for each model
best_results = {}

# Iterate through each model
for model_name, model in models.items():
    print(f"\nEvaluating {model_name}...")
    
    best_accuracy = 0
    best_random_state = None
    accuracy_dict = {}
    
    # Iterate through each random state
    for state in random_states:
        # Check if the model has a random_state parameter
        if 'random_state' in model.get_params():
            model.set_params(random_state=state)

        # Train the model
        model.fit(X_train, y_train)
        
        # Predict on the test set
        y_pred = model.predict(X_test)
        
        # Calculate accuracy
        accuracy = accuracy_score(y_test, y_pred)
        accuracy_dict[state] = accuracy
        
        # Update best accuracy and random state if needed
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_random_state = state
        
        # Print accuracy for the current random state
        print(f"Random State {state}: Accuracy = {accuracy:.2%}")

    # Store the best random state and accuracy for the current model
    best_results[model_name] = {
        'Best Random State': best_random_state,
        'Best Accuracy': best_accuracy,
        'All Accuracies': accuracy_dict
    }

    # Print the best random state and accuracy for the current model
    print(f"\nBest Random State for {model_name}: {best_random_state}")
    print(f"Best Accuracy for {model_name}: {best_accuracy:.2%}")

# Print summary of best results for all models
print("\nSummary of Best Results:")
for model_name, results in best_results.items():
    print(f"{model_name}: Best Random State = {results['Best Random State']}, Best Accuracy = {results['Best Accuracy']:.2%}")



Evaluating Random Forest...
Random State 0: Accuracy = 94.55%
Random State 1: Accuracy = 94.55%
Random State 2: Accuracy = 95.00%
Random State 3: Accuracy = 94.55%
Random State 4: Accuracy = 94.55%
Random State 5: Accuracy = 94.09%
Random State 6: Accuracy = 94.55%
Random State 7: Accuracy = 94.55%
Random State 8: Accuracy = 94.55%
Random State 9: Accuracy = 94.09%
Random State 10: Accuracy = 94.55%
Random State 11: Accuracy = 95.00%
Random State 12: Accuracy = 94.55%
Random State 13: Accuracy = 95.00%
Random State 14: Accuracy = 95.00%
Random State 15: Accuracy = 94.55%
Random State 16: Accuracy = 94.55%
Random State 17: Accuracy = 94.09%
Random State 18: Accuracy = 94.55%
Random State 19: Accuracy = 94.55%
Random State 20: Accuracy = 94.09%
Random State 21: Accuracy = 95.45%
Random State 22: Accuracy = 94.55%
Random State 23: Accuracy = 95.45%
Random State 24: Accuracy = 95.00%
Random State 25: Accuracy = 95.00%
Random State 26: Accuracy = 94.55%
Random State 27: Accuracy = 95.45%
R

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score, average_precision_score
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import LabelEncoder
# Assuming best parameters are already defined:
# best_xgb_params, best_rf_params, best_knn_params, best_svm_params
le=LabelEncoder()
y_train=le.fit_transform(y_train)
y_test=le.fit_transform(y_test)
# Define the range of random states to test
random_states = range(0, 50)  # Adjust this range as needed

best_knn_params={'n_neighbors': 7, 'metric': 'euclidean', 'weights': 'distance', 'p': 2}
# Define the models with best parameters
models = {
    "XGBoost": XGBClassifier(random_state=0),
    "Random Forest": RandomForestClassifier(random_state=21),
    "KNN": KNeighborsClassifier(**best_knn_params),
    "SVM": SVC(probability=True)  # Using probability=True for ROC/PR-AUC
}

# Binarize the true labels for PR-AUC and ROC-AUC calculations (required for multiclass)
y_test_binarized = label_binarize(y_test, classes=np.unique(y_test))

# Dictionary to store metrics for each model
metrics = {}

# Evaluate each model
for model_name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    
    # Predict classes
    y_pred = model.predict(X_test)
    
    # Predict probabilities for AUC metrics
    if hasattr(model, "predict_proba"):
        y_pred_proba = model.predict_proba(X_test)
    else:  # For models like SVM without `predict_proba`, use decision_function
        y_pred_proba = model.decision_function(X_test)
    
    # Calculate metrics
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    accuracy = accuracy_score(y_test, y_pred)
    
    # Calculate PR-AUC and ROC-AUC
    pr_auc = average_precision_score(y_test_binarized, y_pred_proba, average="macro")
    roc_auc = roc_auc_score(y_test_binarized, y_pred_proba, multi_class="ovr", average="macro")
    
    # Store metrics for the current model
    metrics[model_name] = {
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1,
        "Accuracy": accuracy,
        "PR-AUC": pr_auc,
        "ROC-AUC": roc_auc
    }

# Print metrics for each model
for model_name, metric_values in metrics.items():
    print(f"\nMetrics for {model_name}:")
    for metric_name, value in metric_values.items():
        print(f"  {metric_name}: {value:.2%}")



Metrics for XGBoost:
  Precision: 94.72%
  Recall: 95.00%
  F1-Score: 94.79%
  Accuracy: 95.00%
  PR-AUC: 87.95%
  ROC-AUC: 97.00%

Metrics for Random Forest:
  Precision: 95.26%
  Recall: 95.45%
  F1-Score: 95.33%
  Accuracy: 95.45%
  PR-AUC: 88.69%
  ROC-AUC: 96.91%

Metrics for KNN:
  Precision: 95.88%
  Recall: 95.91%
  F1-Score: 95.67%
  Accuracy: 95.91%
  PR-AUC: 90.55%
  ROC-AUC: 97.81%

Metrics for SVM:
  Precision: 91.74%
  Recall: 90.45%
  F1-Score: 87.83%
  Accuracy: 90.45%
  PR-AUC: 70.64%
  ROC-AUC: 91.59%


In [ ]:
import optuna
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

# Split the data into train and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

# # Standardize the features (important for KNN)
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# Define the Optuna objective function
def objective(trial):
    # Hyperparameters to tune
    n_neighbors = trial.suggest_int('n_neighbors', 3, 15)  # Number of neighbors
    metric = trial.suggest_categorical('metric', ['euclidean', 'manhattan', 'minkowski'])  # Distance metric
    weights = trial.suggest_categorical('weights', ['uniform', 'distance'])  # Weight function
    p = trial.suggest_int('p', 1, 2)  # Power parameter for Minkowski distance (1 for Manhattan, 2 for Euclidean)

    # Create the KNN model with the hyperparameters
    model = KNeighborsClassifier(n_neighbors=n_neighbors, metric=metric, weights=weights, p=p)

    # Train the model
    model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = model.predict(X_test)

    # Evaluate the accuracy
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy  # Return the accuracy as the objective to maximize

# Create an Optuna study and optimize the hyperparameters
study = optuna.create_study(direction='maximize')  # Maximize accuracy
study.optimize(objective, n_trials=5000)  # Number of trials can be adjusted based on time and computation

# Print the best hyperparameters and corresponding accuracy
print(f"Best Hyperparameters: {study.best_params}")
print(f"Best Accuracy: {study.best_value:.4f}")

# Optionally, you can train the model with the best parameters
best_params = study.best_params
best_knn = KNeighborsClassifier(n_neighbors=best_params['n_neighbors'],
                                metric=best_params['metric'],
                                weights=best_params['weights'],
                                p=best_params['p'])

best_knn.fit(X_train, y_train)
y_pred_best_knn = best_knn.predict(X_test)

# Final Evaluation
accuracy_best_knn = accuracy_score(y_test, y_pred_best_knn)
print(f"Accuracy with Optimized KNN: {accuracy_best_knn * 100:.2f}%")


[I 2024-11-25 22:20:34,353] A new study created in memory with name: no-name-9eee66e8-832a-41b5-a3d4-c22f6830a3aa
[I 2024-11-25 22:20:34,362] Trial 0 finished with value: 0.95 and parameters: {'n_neighbors': 4, 'metric': 'euclidean', 'weights': 'distance', 'p': 1}. Best is trial 0 with value: 0.95.
[I 2024-11-25 22:20:34,367] Trial 1 finished with value: 0.9545454545454546 and parameters: {'n_neighbors': 8, 'metric': 'euclidean', 'weights': 'distance', 'p': 1}. Best is trial 1 with value: 0.9545454545454546.
[I 2024-11-25 22:20:34,374] Trial 2 finished with value: 0.9409090909090909 and parameters: {'n_neighbors': 13, 'metric': 'manhattan', 'weights': 'distance', 'p': 1}. Best is trial 1 with value: 0.9545454545454546.
[I 2024-11-25 22:20:34,380] Trial 3 finished with value: 0.9454545454545454 and parameters: {'n_neighbors': 12, 'metric': 'manhattan', 'weights': 'distance', 'p': 1}. Best is trial 1 with value: 0.9545454545454546.
[I 2024-11-25 22:20:34,391] Trial 4 finished with value:

Best Hyperparameters: {'n_neighbors': 7, 'metric': 'euclidean', 'weights': 'distance', 'p': 2}
Best Accuracy: 0.9591
Accuracy with Optimized KNN: 95.91%


In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score, average_precision_score
from sklearn.preprocessing import label_binarize

# Assuming best parameters are already defined:
# best_xgb_params, best_rf_params, best_knn_params, best_svm_params
best_knn_params={'n_neighbors': 7, 'metric': 'euclidean', 'weights': 'distance', 'p': 2}
# Define the models with best parameters
models = {
    "XGBoost": XGBClassifier(random_state=1),
    "Random Forest": RandomForestClassifier(random_state=21),
    "KNN": KNeighborsClassifier(**best_knn_params),
    "SVM": SVC(probability=True)  # Using probability=True for ROC/PR-AUC
}

# Binarize the true labels for PR-AUC and ROC-AUC calculations (required for multiclass)
y_test_binarized = label_binarize(y_test, classes=np.unique(y_test))

# Dictionary to store metrics for each model
metrics = {}

# Evaluate each model
for model_name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    
    # Predict classes
    y_pred = model.predict(X_test)
    
    # Predict probabilities for AUC metrics
    if hasattr(model, "predict_proba"):
        y_pred_proba = model.predict_proba(X_test)
    else:  # For models like SVM without `predict_proba`, use decision_function
        y_pred_proba = model.decision_function(X_test)
    
    # Calculate metrics
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    accuracy = accuracy_score(y_test, y_pred)
    
    # Calculate PR-AUC and ROC-AUC
    pr_auc = average_precision_score(y_test_binarized, y_pred_proba, average="macro")
    roc_auc = roc_auc_score(y_test_binarized, y_pred_proba, multi_class="ovr", average="macro")
    
    # Store metrics for the current model
    metrics[model_name] = {
        "Precision": precision,
        "Recall": recall,
        "F1-Score": f1,
        "Accuracy": accuracy,
        "PR-AUC": pr_auc,
        "ROC-AUC": roc_auc
    }

# Print metrics for each model
for model_name, metric_values in metrics.items():
    print(f"\nMetrics for {model_name}:")
    for metric_name, value in metric_values.items():
        print(f"  {metric_name}: {value:.2%}")



Metrics for XGBoost:
  Precision: 94.72%
  Recall: 95.00%
  F1-Score: 94.79%
  Accuracy: 95.00%
  PR-AUC: 87.95%
  ROC-AUC: 97.00%

Metrics for Random Forest:
  Precision: 95.26%
  Recall: 95.45%
  F1-Score: 95.33%
  Accuracy: 95.45%
  PR-AUC: 88.69%
  ROC-AUC: 96.91%

Metrics for KNN:
  Precision: 95.88%
  Recall: 95.91%
  F1-Score: 95.67%
  Accuracy: 95.91%
  PR-AUC: 90.55%
  ROC-AUC: 97.81%

Metrics for SVM:
  Precision: 91.74%
  Recall: 90.45%
  F1-Score: 87.83%
  Accuracy: 90.45%
  PR-AUC: 70.64%
  ROC-AUC: 91.58%
